In [1]:
!nvidia-smi

Mon Jul  4 16:12:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/My\ Drive/Licenta/GAN/CycleGAN
!ls

/content/drive/My Drive/Licenta/GAN/CycleGAN
CycleGAN_FID.ipynb	      FID_outputs   prev_versions
CycleGAN_IS.ipynb	      mountain.jpg  startegy_comparison.gdoc
cyclegan_resnet_gen.ipynb     ouputs	    test_CycleGAN
cyclegan_resnet_gen_v2.ipynb  photo_jpg


### Import libraries

In [4]:
import glob
import cv2
from PIL import Image
import numpy as np
from scipy import linalg
import tensorflow as tf
from tensorflow import keras

### Define functions needed for computing IS

In [5]:
def inception_score(pred, num_examples = 7038, splits = 10):
  
  # define list to store the inception scores
  IS_list = []

  # compute the number of elements in each split
  num_elem = int(np.floor(num_examples / splits))

  for i in range(splits):

    # define split start and split end
    split_start, split_end = i * num_elem, i * num_elem + num_elem

    # obtain conditional probability p(y|x)
    cond_prob = pred[split_start:split_end]

    # compute marginal probability p(y)
    marg_prob = np.expand_dims(cond_prob.mean(axis = 0), 0)

    # compute Kullback-Leibler divergence
    KL_div = cond_prob * (np.log(cond_prob) - np.log(marg_prob))

    # IS is obtained by summing over the classes, averaging and applying exponential
    IS = np.exp(np.mean(KL_div.sum(axis=1)))

    # append the obtained inception score to the list
    IS_list.append(IS)

  # return mean and standard deviation of obtained scores
  return np.mean(IS_list), np.std(IS_list)

In [6]:
#@title Choose art style

style = 'Ukiyoe' #@param ["Cezanne", "Monet", "Ukiyoe", "VanGogh"]

### Read the real and generated images

In [7]:
generated_art_paths = glob.glob(f'FID_outputs/{style.lower()}/generated_art/*.jpg')
generated_art_dataset = np.array([np.array(Image.open(generated_art_path)) for generated_art_path in generated_art_paths])
print(f"Loaded generated art dataset of shape = {generated_art_dataset.shape}")

Loaded generated art dataset of shape = (7038, 256, 256, 3)


### Preprocess data

In [8]:
# convert to 32-bit floats
generated_art_dataset = generated_art_dataset.astype('float32')

In [9]:
# resize images in generated art dataset to (299, 299, 3)
generated_art_dataset = np.asarray([cv2.resize(generated_art, (299, 299)) for generated_art in generated_art_dataset])

In [10]:
# preprocess images in generated art dataset so that they can be applied to InceptionV3 model
generated_art_dataset = keras.applications.inception_v3.preprocess_input(generated_art_dataset)

### Load model and get predictions

In [11]:
# load InceptionV3
model = keras.applications.inception_v3.InceptionV3()

96124928/96112376 [==============================] - 1s 0us/step


In [12]:
# get InceptionV3 prediction
pred = model.predict(generated_art_dataset)

### Compute IS

In [13]:
num_examples = generated_art_dataset.shape[0]
IS_mean, IS_std = inception_score(pred)
print(f"Inception Score Mean = {IS_mean}, Inception Score Standard Deviation = {IS_std}")

Inception Score Mean = 5.721205234527588, Inception Score Standard Deviation = 0.23070980608463287
